# Ovation Aurora Short Term Forecast
Short notebook ingesting text file, and saving as pandas array, along with forecast timestamp.

Link: http://services.swpc.noaa.gov/text/aurora-nowcast-map.txt

On the website, service looks like:
<table>
    <tr>
        <th>Northern Hemisphere</th>
        <th>Southern Hemisphere</th>
    </tr>
    <tr>
        <td><img src="http://services.swpc.noaa.gov/images/aurora-forecast-northern-hemisphere.png" width=300></td>
        <td><img src="http://services.swpc.noaa.gov/images/aurora-forecast-southern-hemisphere.png" width=300></td>
    </tr>
</table>

In [15]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

# Will need to fix the connection to get the update every n minutes,
# but for now devputer can't connect to the Internet.
def ovation_forecast():
    """Retrieves the OVATION aurora forecast from the swpc.noaa site. If
    Internet is down, retrieves local copy.
    Extent:
    1024 values covering 0 to 360 degrees in longitude (0.32846715 deg/px)
    512 values covering -90 to 90 degrees in latitude (0.3515625 deg/px)
    Returns:
    array: pandas array of the forecast
    datetime: date & time of forecast
    None -> np.array, datetime"""
    img_extent = (-180, 180, -90, 90)
    url = 'http://services.swpc.noaa.gov/text/aurora-nowcast-map.txt'
    loc_copy = 'aurora-nowcast-map.txt'
    timestamp = None
    
    def connected_to_internet(host='8.8.8.8', port=53, timeout=1):
        """Host: 8.8.8.8 (google-public-dns-a.google.com)
        OpenPort: 53/tcp
        Service: domain (DNS/TCP)
        See http://stackoverflow.com/questions/3764291/checking-network-connection
        """
        import socket
        try:
            socket.setdefaulttimeout(timeout)
            socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect((host, port))
            return True
        except Exception as ex:
            print(ex, ", using local copy instead")
            return False
    
    if connected_to_internet():
        file = requests.get(url, timeout=1).read()
    else:
        file = loc_copy
    
    # Get timestamp
    with open(file) as f:
        for line in f:
            if line.startswith('# Product Valid At:'):
                timestamp = datetime.strptime(line[-17:-1], '%Y-%m-%d %H:%M')

    # Get data grid
    # [ ] TODO: find a way to only open file once.
    array = pd.read_csv(file, comment='#', delimiter='\s+', header=None)
    
    return array, timestamp


# Low-tech unit testing
_array, _timestamp = ovation_forecast()
assert _array.shape == (512, 1024)
assert str(_timestamp)[0:2] == '20'  # From '2017...'

timed out , using local copy instead


## Visualization
### Equirectangular Projection (Plate Carrée)
We'll start off with a basic matplotlib figure, where each pixel in the data file represents approximately a third of a degree. The figure below, however, has been expanded a bit, with bicubic interpolation to smooth the values between the dots. With proper transparency, this could be overlaid on top of the base map in Google Earth.

In [ ]:
data = ovation_forecast()[0]
fig = plt.subplots(figsize=(20, 20))
plt.imshow(data, 
           interpolation='bicubic', 
           cmap='magma', 
           origin='lower',
          )
plt.colorbar(shrink=0.4)
plt.xticks([]), plt.yticks([])
plt.show()

### Polar Projection
To recreate the two figures in the introduction, we need to re-project it accordingly. For this we use cartopy, provided by the [British Met Office](http://scitools.org.uk/cartopy/). In their own words, some of the key features of cartopy are:

* object oriented projection definitions
* point, line, vector, polygon and image transformations between projections
* integration to expose advanced mapping in matplotlib with a simple and intuitive interface
* powerful vector data handling by integrating shapefile reading with Shapely capabilities


    [ ] TODO: Look into incorporating Shapely for extra types of data for export.


In [ ]:
import cartopy

